# 🏥 Comprehensive Exploratory Data Analysis
## Biomechanical Features of Orthopedic Patients

### 📋 Executive Summary
This notebook provides a comprehensive exploratory data analysis of biomechanical features in orthopedic patients. The analysis includes statistical tests, advanced visualizations, dimensionality reduction, and clustering techniques to understand the relationship between biomechanical measurements and orthopedic conditions.

### 🎯 Key Objectives
1. **Dataset Overview**: Understand data structure, missing values, and basic statistics
2. **Target Analysis**: Examine class distribution and balance
3. **Feature Analysis**: Analyze numerical features, distributions, and outliers
4. **Correlation Analysis**: Identify multicollinearity and feature relationships
5. **Statistical Testing**: Perform hypothesis testing for group differences
6. **Dimensionality Reduction**: Apply PCA and t-SNE for visualization
7. **Clustering**: Discover natural groupings in the data

### 📊 Dataset Information
- **Source**: UCI Machine Learning Repository
- **Features**: 6 biomechanical attributes
- **Target**: 3-class (Normal, Hernia, Spondylolisthesis) and binary classification
- **Size**: 310 patient records

### Comprehensive Exploratory Data Analysis
##### Biomechanical Features of Orthopedic Patients
This notebook provides a comprehensive exploratory data analysis of biomechanical features in orthopedic patients, including statistical tests, visualizations, and dimensionality reduction techniques.

In [ ]:
# Configuration Parameters
ANALYSIS_CONFIG = {
    'random_state': 42,
    'test_size': 0.2,
    'significance_level': 0.05,
    'correlation_threshold': 0.8,
    'pca_n_components': 2,
    'tsne_n_components': 2,
    'tsne_perplexity': 30,
    'umap_n_components': 2,
    'umap_n_neighbors': 15,
    'figure_size': (12, 8),
    'plot_dpi': 300,
    'save_plots': True,
    'plots_folder': 'plots'
}

print("Configuration parameters loaded successfully!")
print("Available parameters:")
for key, value in ANALYSIS_CONFIG.items():
    print(f"  {key}: {value}")

In [ ]:
# Custom output handler for better visualization
class MultiOut:
    def __init__(self, *outputs):
        self.outputs = outputs
    
    def write(self, s):
        for out in self.outputs:
            out.write(s)
    
    def flush(self):
        for out in self.outputs:
            out.flush()

print("MultiOut class defined successfully!")

### 🎛️ Analysis Configuration
You can modify these parameters to customize the analysis:

In [ ]:
# Analysis Configuration Parameters
ANALYSIS_CONFIG = {
    'correlation_threshold': 0.7,  # Threshold for high correlation
    'outlier_z_threshold': 3.0,    # Z-score threshold for outliers
    'significance_level': 0.05,    # P-value threshold for statistical tests
    'pca_variance_threshold': 0.95, # Variance threshold for PCA
    'figure_size': (15, 10),       # Default figure size
    'save_plots': True,            # Whether to save plots
    'plot_format': 'png',          # Plot save format
    'random_state': 42             # Random state for reproducibility
}

print("📋 Analysis Configuration:")
for key, value in ANALYSIS_CONFIG.items():
    print(f"  {key}: {value}")

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import levene
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import warnings
import os
import sys
import kagglehub
import umap

# Set up plotting configuration for inline display
%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = (12, 8)

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("All libraries imported successfully!")
print("Available packages:")
print("- pandas, numpy, matplotlib, seaborn")
print("- scipy, sklearn, umap")
print("- kagglehub for data access")
print("- warnings and os for system operations")
print("📊 Matplotlib configured for inline display + file saving")

In [ ]:
# Create plots directory and load data
plots_folder = ANALYSIS_CONFIG['plots_folder']
if not os.path.exists(plots_folder):
    os.makedirs(plots_folder)
    print(f"Created plots directory: {plots_folder}")
else:
    print(f"Plots directory already exists: {plots_folder}")

def load_data():
    """Load the Vertebral Column dataset"""
    try:
        # Download latest version
        path = kagglehub.dataset_download("sammy123/lower-back-pain-symptoms-dataset")
        print(f"Dataset downloaded to: {path}")
        
        # Load the dataset
        data_path = os.path.join(path, "Dataset_spine.csv")
        df = pd.read_csv(data_path)
        print(f"Dataset loaded successfully! Shape: {df.shape}")
        return df
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

# Load the data immediately
print("🚀 Loading dataset...")
df = load_data()

if df is not None:
    print("✅ Dataset loaded successfully!")
    print(f"📊 Dataset shape: {df.shape}")
    print(f"📋 Columns: {list(df.columns)}")
else:
    print("❌ Failed to load dataset. Please check the data source.")
    
print("\n🔧 Data loading complete!")

## 2. Dataset Overview Function

In [ ]:
def dataset_overview(df):
    """
    Provide comprehensive overview of the dataset including:
    - Basic information (shape, columns, data types)
    - Missing values analysis
    - Basic statistics
    - Data quality assessment
    """
    print("="*50)
    print("1. DATASET OVERVIEW")
    print("="*50)
    
    print(f"Dataset Shape: {df.shape}")
    print(f"Features: {df.shape[1]}")
    print(f"Samples: {df.shape[0]}")
    
    print("\nColumn Information:")
    print(df.info())
    
    print("\nData Types:")
    print(df.dtypes.value_counts())
    
    print("\nMissing Values:")
    missing_values = df.isnull().sum()
    if missing_values.sum() > 0:
        print(missing_values[missing_values > 0])
    else:
        print("No missing values found!")
    
    print("\nFirst 5 rows:")
    print(df.head())
    
    print("\nLast 5 rows:")
    print(df.tail())
    
    print("\nBasic Statistics:")
    print(df.describe())
    
    # Check for duplicates
    duplicates = df.duplicated().sum()
    print(f"\nDuplicate rows: {duplicates}")
    
    # Memory usage
    print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Execute immediately
if df is not None:
    dataset_overview(df)
else:
    print("❌ No data available for analysis")

## 3. Target Variable Analysis Function

In [ ]:
def target_analysis(df, target_col='class'):
    """
    Analyze the target variable including:
    - Value counts and proportions
    - Visualization of class distribution
    - Class balance assessment
    """
    print("\n" + "="*50)
    print("2. TARGET VARIABLE ANALYSIS")
    print("="*50)
    
    # Value counts and proportions
    value_counts = df[target_col].value_counts()
    proportions = df[target_col].value_counts(normalize=True)
    
    print(f"Target variable: {target_col}")
    print(f"Unique classes: {df[target_col].nunique()}")
    print("\nClass Distribution:")
    
    for class_name, count in value_counts.items():
        percentage = proportions[class_name] * 100
        print(f"  {class_name}: {count} ({percentage:.1f}%)")
    
    # Visualizations
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Count plot
    sns.countplot(data=df, x=target_col, ax=axes[0])
    axes[0].set_title('Class Distribution (Count)')
    axes[0].set_xlabel('Class')
    axes[0].set_ylabel('Count')
    
    # Add count labels on bars
    for i, v in enumerate(value_counts.values):
        axes[0].text(i, v + 0.5, str(v), ha='center', va='bottom')
    
    # Pie chart
    axes[1].pie(value_counts.values, labels=value_counts.index, autopct='%1.1f%%', startangle=90)
    axes[1].set_title('Class Distribution (Proportion)')
    
    plt.tight_layout()
    plt.savefig(f"{plots_folder}/target_distribution_{target_col}.png", bbox_inches="tight")
    plt.show()
    
    # Class balance assessment
    max_class = value_counts.max()
    min_class = value_counts.min()
    balance_ratio = min_class / max_class
    
    print(f"\nClass Balance Assessment:")
    print(f"Most frequent class: {value_counts.idxmax()} ({max_class} samples)")
    print(f"Least frequent class: {value_counts.idxmin()} ({min_class} samples)")
    print(f"Balance ratio: {balance_ratio:.3f}")
    
    if balance_ratio < 0.5:
        print("⚠️  Dataset is imbalanced - consider using stratified sampling or class weighting")
    else:
        print("✅ Dataset is relatively balanced")

# Execute immediately
if df is not None:
    target_analysis(df, target_col='class')
else:
    print("❌ No data available for analysis")

## 4. Numerical Features Analysis Function

In [ ]:
def numerical_features_analysis(df, target_col='class'):
    """
    Analyze numerical features including:
    - Distribution plots
    - Box plots by class
    - Statistical summaries by class
    """
    print("\n" + "="*50)
    print("3. NUMERICAL FEATURES ANALYSIS")
    print("="*50)
    
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    print(f"Numerical features: {len(numerical_cols)}")
    print(f"Features: {numerical_cols}")
    
    # Distribution plots
    n_cols = 3
    n_rows = (len(numerical_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, col in enumerate(numerical_cols):
        row = i // n_cols
        col_idx = i % n_cols
        
        axes[row, col_idx].hist(df[col], bins=30, alpha=0.7, edgecolor='black')
        axes[row, col_idx].set_title(f'Distribution of {col}')
        axes[row, col_idx].set_xlabel(col)
        axes[row, col_idx].set_ylabel('Frequency')
        axes[row, col_idx].grid(True, alpha=0.3)
    
    # Remove empty subplots
    for i in range(len(numerical_cols), n_rows * n_cols):
        row = i // n_cols
        col_idx = i % n_cols
        axes[row, col_idx].remove()
    
    plt.tight_layout()
    plt.savefig(f"{plots_folder}/numerical_features_distribution_{target_col}.png", bbox_inches="tight")
    plt.show()
    
    # Box plots by class
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, col in enumerate(numerical_cols):
        row = i // n_cols
        col_idx = i % n_cols
        
        sns.boxplot(data=df, x=target_col, y=col, ax=axes[row, col_idx])
        axes[row, col_idx].set_title(f'{col} by {target_col}')
        axes[row, col_idx].grid(True, alpha=0.3)
    
    # Remove empty subplots
    for i in range(len(numerical_cols), n_rows * n_cols):
        row = i // n_cols
        col_idx = i % n_cols
        axes[row, col_idx].remove()
    
    plt.tight_layout()
    plt.savefig(f"{plots_folder}/numerical_features_boxplots_{target_col}.png", bbox_inches="tight")
    plt.show()
    
    # Statistical summaries by class
    print("\nStatistical Summary by Class:")
    for class_name in df[target_col].unique():
        print(f"\n{class_name}:")
        print(df[df[target_col] == class_name][numerical_cols].describe())

# Execute immediately
if df is not None:
    numerical_features_analysis(df, target_col='class')
else:
    print("❌ No data available for analysis")

## 5. Outlier Analysis Function

In [ ]:
def outlier_analysis(df, target_col='class'):
    """
    Detect and analyze outliers using:
    - IQR method
    - Z-score method
    - Isolation Forest
    - Visualization of outliers
    """
    print("\n" + "="*50)
    print("4. OUTLIER DETECTION AND ANALYSIS")
    print("="*50)
    
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    outlier_summary = {}
    
    for col in numerical_cols:
        print(f"\nAnalyzing outliers in {col}:")
        
        # IQR method
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        iqr_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        
        # Z-score method
        z_scores = np.abs(stats.zscore(df[col]))
        z_outliers = df[z_scores > 3]
        
        print(f"  IQR method: {len(iqr_outliers)} outliers ({len(iqr_outliers)/len(df)*100:.1f}%)")
        print(f"  Z-score method: {len(z_outliers)} outliers ({len(z_outliers)/len(df)*100:.1f}%)")
        print(f"  IQR bounds: [{lower_bound:.2f}, {upper_bound:.2f}]")
        
        outlier_summary[col] = {
            'IQR_outliers': len(iqr_outliers),
            'Z_outliers': len(z_outliers),
            'IQR_percentage': len(iqr_outliers)/len(df)*100,
            'Z_percentage': len(z_outliers)/len(df)*100
        }
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Box plot for all features
    df_scaled = df[numerical_cols].copy()
    for col in numerical_cols:
        df_scaled[col] = (df_scaled[col] - df_scaled[col].mean()) / df_scaled[col].std()
    
    axes[0, 0].boxplot([df_scaled[col] for col in numerical_cols])
    axes[0, 0].set_xticklabels(numerical_cols, rotation=45)
    axes[0, 0].set_title('Box Plot (Standardized Features)')
    axes[0, 0].set_ylabel('Standardized Values')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Outlier count by method
    methods = ['IQR', 'Z-score']
    iqr_counts = [outlier_summary[col]['IQR_outliers'] for col in numerical_cols]
    z_counts = [outlier_summary[col]['Z_outliers'] for col in numerical_cols]
    
    x = np.arange(len(numerical_cols))
    width = 0.35
    
    axes[0, 1].bar(x - width/2, iqr_counts, width, label='IQR', alpha=0.7)
    axes[0, 1].bar(x + width/2, z_counts, width, label='Z-score', alpha=0.7)
    axes[0, 1].set_xlabel('Features')
    axes[0, 1].set_ylabel('Number of Outliers')
    axes[0, 1].set_title('Outlier Count by Method')
    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(numerical_cols, rotation=45)
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Outlier percentage
    iqr_percentages = [outlier_summary[col]['IQR_percentage'] for col in numerical_cols]
    z_percentages = [outlier_summary[col]['Z_percentage'] for col in numerical_cols]
    
    axes[1, 0].bar(x - width/2, iqr_percentages, width, label='IQR', alpha=0.7)
    axes[1, 0].bar(x + width/2, z_percentages, width, label='Z-score', alpha=0.7)
    axes[1, 0].set_xlabel('Features')
    axes[1, 0].set_ylabel('Percentage of Outliers')
    axes[1, 0].set_title('Outlier Percentage by Method')
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels(numerical_cols, rotation=45)
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Summary heatmap
    summary_df = pd.DataFrame(outlier_summary).T
    sns.heatmap(summary_df[['IQR_percentage', 'Z_percentage']], 
                annot=True, fmt='.1f', cmap='Reds', ax=axes[1, 1])
    axes[1, 1].set_title('Outlier Percentage Heatmap')
    axes[1, 1].set_ylabel('Features')
    
    plt.tight_layout()
    plt.savefig(f"{plots_folder}/outlier_detection_summary.png", bbox_inches="tight")
    plt.show()
    
    # Overall summary
    print("\nOutlier Summary:")
    summary_df = pd.DataFrame(outlier_summary).T
    print(summary_df.round(2))

# Execute immediately
if df is not None:
    outlier_analysis(df, target_col='class')
else:
    print("❌ No data available for analysis")

## 6. Correlation Analysis Function

In [ ]:
def correlation_analysis(df, target_col='class'):
    """
    Analyze correlations between features:
    - Correlation matrix
    - Heatmap visualization
    - Identify highly correlated features
    - Multicollinearity assessment
    """
    print("\n" + "="*50)
    print("5. CORRELATION ANALYSIS")
    print("="*50)
    
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    corr_matrix = df[numerical_cols].corr()
    
    print(f"Correlation matrix shape: {corr_matrix.shape}")
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Standard correlation heatmap
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
                cmap='RdBu_r', center=0, ax=axes[0, 0])
    axes[0, 0].set_title('Correlation Matrix (Lower Triangle)')
    
    # Clustermap for grouping similar correlations
    corr_linkage = sns.clustermap(corr_matrix, method='ward', cmap='RdBu_r', 
                                 center=0, figsize=(10, 8), 
                                 cbar_pos=(0.02, 0.8, 0.05, 0.15))
    axes[0, 1].set_title('Correlation Clustermap')
    axes[0, 1].axis('off')
    
    # High correlation pairs
    threshold = ANALYSIS_CONFIG['correlation_threshold']
    high_corr_pairs = []
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_val = corr_matrix.iloc[i, j]
            if abs(corr_val) > threshold:
                high_corr_pairs.append({
                    'Feature1': corr_matrix.columns[i],
                    'Feature2': corr_matrix.columns[j],
                    'Correlation': corr_val
                })
    
    if high_corr_pairs:
        print(f"\nHighly correlated feature pairs (|r| > {threshold}):")
        for pair in high_corr_pairs:
            print(f"  {pair['Feature1']} - {pair['Feature2']}: {pair['Correlation']:.3f}")
        
        # Visualize high correlation pairs
        high_corr_df = pd.DataFrame(high_corr_pairs)
        high_corr_df['abs_correlation'] = high_corr_df['Correlation'].abs()
        
        bars = axes[1, 0].bar(range(len(high_corr_df)), high_corr_df['abs_correlation'])
        axes[1, 0].set_xlabel('Feature Pairs')
        axes[1, 0].set_ylabel('|Correlation|')
        axes[1, 0].set_title(f'High Correlation Pairs (|r| > {threshold})')
        axes[1, 0].set_xticks(range(len(high_corr_df)))
        axes[1, 0].set_xticklabels([f"{pair['Feature1']}-{pair['Feature2']}" for pair in high_corr_pairs], 
                                  rotation=45, ha='right')
        axes[1, 0].axhline(y=threshold, color='red', linestyle='--', alpha=0.7)
        axes[1, 0].grid(True, alpha=0.3)
    else:
        print(f"\nNo highly correlated feature pairs found (|r| > {threshold})")
        axes[1, 0].text(0.5, 0.5, f'No high correlations\n(|r| > {threshold})', 
                       ha='center', va='center', transform=axes[1, 0].transAxes)
        axes[1, 0].set_title('High Correlation Analysis')
    
    # Multicollinearity assessment using VIF
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    
    try:
        vif_data = pd.DataFrame()
        vif_data["Feature"] = numerical_cols
        vif_data["VIF"] = [variance_inflation_factor(df[numerical_cols].values, i) 
                          for i in range(len(numerical_cols))]
        
        # Plot VIF values
        bars = axes[1, 1].bar(range(len(vif_data)), vif_data['VIF'])
        axes[1, 1].set_xlabel('Features')
        axes[1, 1].set_ylabel('VIF Value')
        axes[1, 1].set_title('Variance Inflation Factor (VIF)')
        axes[1, 1].set_xticks(range(len(vif_data)))
        axes[1, 1].set_xticklabels(vif_data['Feature'], rotation=45, ha='right')
        axes[1, 1].axhline(y=5, color='orange', linestyle='--', alpha=0.7, label='Moderate (5)')
        axes[1, 1].axhline(y=10, color='red', linestyle='--', alpha=0.7, label='High (10)')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        print("\nVariance Inflation Factor (VIF) Analysis:")
        print(vif_data.round(2))
        
        high_vif = vif_data[vif_data['VIF'] > 5]
        if not high_vif.empty:
            print(f"\nFeatures with high VIF (>5):")
            print(high_vif)
        else:
            print("\nNo features with high multicollinearity detected (VIF > 5)")
            
    except Exception as e:
        print(f"Error calculating VIF: {e}")
        axes[1, 1].text(0.5, 0.5, 'VIF calculation\nfailed', 
                       ha='center', va='center', transform=axes[1, 1].transAxes)
    
    plt.tight_layout()
    plt.savefig(f"{plots_folder}/correlation_analysis.png", bbox_inches="tight")
    plt.show()
    
    # Save clustermap separately
    plt.figure(figsize=(10, 8))
    sns.clustermap(corr_matrix, method='ward', cmap='RdBu_r', center=0, 
                  cbar_pos=(0.02, 0.8, 0.05, 0.15))
    plt.savefig(f"{plots_folder}/correlation_clustermap.png", bbox_inches="tight")
    plt.show()

# Execute immediately
if df is not None:
    correlation_analysis(df, target_col='class')
else:
    print("❌ No data available for analysis")

## 7. Feature Relationships Function

In [ ]:
def feature_relationships(df, target_col='class'):
    """
    Analyze relationships between features:
    - Pairplot
    - Feature interactions
    - Violin plots by class
    """
    print("\n" + "="*50)
    print("6. FEATURE RELATIONSHIPS")
    print("="*50)
    
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Pairplot (sample if too many features)
    if len(numerical_cols) > 6:
        print(f"Too many features ({len(numerical_cols)}) for pairplot. Showing first 6.")
        cols_for_pairplot = numerical_cols[:6]
    else:
        cols_for_pairplot = numerical_cols
    
    print(f"Creating pairplot for features: {cols_for_pairplot}")
    
    # Create pairplot
    pairplot_df = df[cols_for_pairplot + [target_col]]
    g = sns.pairplot(pairplot_df, hue=target_col, diag_kind='kde', 
                     plot_kws={'alpha': 0.6}, diag_kws={'alpha': 0.7})
    g.fig.suptitle('Feature Pairplot by Class', y=1.02)
    
    plt.savefig(f"{plots_folder}/feature_pairplot.png", bbox_inches="tight")
    plt.show()
    
    # Feature interactions - violin plots
    n_cols = 3
    n_rows = (len(numerical_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, col in enumerate(numerical_cols):
        row = i // n_cols
        col_idx = i % n_cols
        
        sns.violinplot(data=df, x=target_col, y=col, ax=axes[row, col_idx])
        axes[row, col_idx].set_title(f'{col} Distribution by {target_col}')
        axes[row, col_idx].grid(True, alpha=0.3)
    
    # Remove empty subplots
    for i in range(len(numerical_cols), n_rows * n_cols):
        row = i // n_cols
        col_idx = i % n_cols
        axes[row, col_idx].remove()
    
    plt.tight_layout()
    plt.savefig(f"{plots_folder}/feature_interactions_violin_{target_col}.png", bbox_inches="tight")
    plt.show()
    
    # Calculate feature importance using correlation with target
    if target_col in df.columns:
        # Create numerical encoding of target for correlation
        target_encoded = pd.get_dummies(df[target_col], drop_first=True)
        
        if target_encoded.shape[1] == 1:  # Binary classification
            target_numeric = target_encoded.iloc[:, 0]
            correlations = df[numerical_cols].corrwith(target_numeric)
            
            print("\nFeature-Target Correlations:")
            correlations_sorted = correlations.abs().sort_values(ascending=False)
            print(correlations_sorted.round(3))
            
            # Plot feature importance
            plt.figure(figsize=(10, 6))
            correlations_sorted.plot(kind='bar')
            plt.title('Feature Importance (Absolute Correlation with Target)')
            plt.xlabel('Features')
            plt.ylabel('|Correlation|')
            plt.xticks(rotation=45)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig(f"{plots_folder}/feature_importance_correlation.png", bbox_inches="tight")
            plt.show()
        else:
            print("Multi-class target detected - correlation analysis skipped")
    
    print(f"\nFeature relationship analysis completed for {len(numerical_cols)} features")

# Execute immediately
if df is not None:
    feature_relationships(df, target_col='class')
else:
    print("❌ No data available for analysis")

## 8. Dimensionality Reduction Function

In [ ]:
def dimensionality_analysis(df, target_col='class'):
    """
    Perform dimensionality reduction analysis:
    - PCA analysis with explained variance
    - t-SNE visualization
    - UMAP visualization
    """
    print("\n" + "="*50)
    print("7. DIMENSIONALITY REDUCTION ANALYSIS")
    print("="*50)
    
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    X = df[numerical_cols].dropna()
    y = df[target_col]
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # PCA Analysis
    pca = PCA()
    X_pca = pca.fit_transform(X_scaled)
    
    print(f"Original dimensions: {X_scaled.shape}")
    print(f"PCA components: {pca.n_components_}")
    
    # Explained variance
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance_ratio)
    
    print(f"First 2 components explain {cumulative_variance[1]:.3f} of variance")
    print(f"First 3 components explain {cumulative_variance[2]:.3f} of variance")
    
    # Find number of components for 95% variance
    n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
    print(f"Components needed for 95% variance: {n_components_95}")
    
    # t-SNE
    print("\nComputing t-SNE...")
    tsne = TSNE(n_components=ANALYSIS_CONFIG['tsne_n_components'], 
                perplexity=ANALYSIS_CONFIG['tsne_perplexity'],
                random_state=ANALYSIS_CONFIG['random_state'])
    X_tsne = tsne.fit_transform(X_scaled)
    
    # UMAP
    print("Computing UMAP...")
    reducer = umap.UMAP(n_components=ANALYSIS_CONFIG['umap_n_components'],
                       n_neighbors=ANALYSIS_CONFIG['umap_n_neighbors'],
                       random_state=ANALYSIS_CONFIG['random_state'])
    X_umap = reducer.fit_transform(X_scaled)
    
    # Visualizations
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # PCA explained variance
    axes[0, 0].bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio)
    axes[0, 0].set_xlabel('Principal Component')
    axes[0, 0].set_ylabel('Explained Variance Ratio')
    axes[0, 0].set_title('PCA Explained Variance')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Cumulative explained variance
    axes[0, 1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'bo-')
    axes[0, 1].axhline(y=0.95, color='red', linestyle='--', alpha=0.7, label='95%')
    axes[0, 1].set_xlabel('Number of Components')
    axes[0, 1].set_ylabel('Cumulative Explained Variance')
    axes[0, 1].set_title('Cumulative Explained Variance')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # PCA visualization
    scatter = axes[0, 2].scatter(X_pca[:, 0], X_pca[:, 1], c=y.astype('category').cat.codes, 
                                cmap='viridis', alpha=0.6)
    axes[0, 2].set_xlabel(f'PC1 ({explained_variance_ratio[0]:.3f})')
    axes[0, 2].set_ylabel(f'PC2 ({explained_variance_ratio[1]:.3f})')
    axes[0, 2].set_title('PCA Visualization')
    plt.colorbar(scatter, ax=axes[0, 2])
    
    # t-SNE visualization
    scatter = axes[1, 0].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y.astype('category').cat.codes, 
                                cmap='viridis', alpha=0.6)
    axes[1, 0].set_xlabel('t-SNE 1')
    axes[1, 0].set_ylabel('t-SNE 2')
    axes[1, 0].set_title('t-SNE Visualization')
    plt.colorbar(scatter, ax=axes[1, 0])
    
    # UMAP visualization
    scatter = axes[1, 1].scatter(X_umap[:, 0], X_umap[:, 1], c=y.astype('category').cat.codes, 
                                cmap='viridis', alpha=0.6)
    axes[1, 1].set_xlabel('UMAP 1')
    axes[1, 1].set_ylabel('UMAP 2')
    axes[1, 1].set_title('UMAP Visualization')
    plt.colorbar(scatter, ax=axes[1, 1])
    
    # Feature loadings for first two PCs
    loadings = pca.components_[:2].T
    axes[1, 2].scatter(loadings[:, 0], loadings[:, 1], alpha=0.7)
    for i, feature in enumerate(numerical_cols):
        axes[1, 2].annotate(feature, (loadings[i, 0], loadings[i, 1]), 
                           xytext=(5, 5), textcoords='offset points', fontsize=8)
    axes[1, 2].set_xlabel('PC1 Loading')
    axes[1, 2].set_ylabel('PC2 Loading')
    axes[1, 2].set_title('Feature Loadings (PC1 vs PC2)')
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"{plots_folder}/dimensionality_reduction_{target_col}.png", bbox_inches="tight")
    plt.show()
    
    # Feature importance from PCA
    print("\nFeature importance from first 2 PCs:")
    feature_importance = np.abs(loadings).mean(axis=1)
    feature_importance_df = pd.DataFrame({
        'Feature': numerical_cols,
        'Importance': feature_importance
    }).sort_values('Importance', ascending=False)
    
    print(feature_importance_df.round(3))

# Execute immediately
if df is not None:
    dimensionality_analysis(df, target_col='class')
else:
    print("❌ No data available for analysis")

## 9. Clustering Analysis Function

In [ ]:
def clustering_analysis(df, target_col='class'):
    """
    Perform clustering analysis:
    - K-means clustering with optimal k selection
    - Elbow method for k selection
    - Silhouette analysis
    - Comparison with true labels
    """
    print("\n" + "="*50)
    print("8. CLUSTERING ANALYSIS")
    print("="*50)
    
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    X = df[numerical_cols].dropna()
    
    # Standardize features for clustering
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Elbow method: calculate inertia for different k values
    inertias = []
    k_range = range(1, 11)
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=ANALYSIS_CONFIG['random_state'], n_init=10)
        kmeans.fit(X_scaled)
        inertias.append(kmeans.inertia_)
    
    # Silhouette analysis for optimal k
    silhouette_scores = []
    for k in range(2, 11):
        kmeans = KMeans(n_clusters=k, random_state=ANALYSIS_CONFIG['random_state'], n_init=10)
        cluster_labels = kmeans.fit_predict(X_scaled)
        silhouette_avg = silhouette_score(X_scaled, cluster_labels)
        silhouette_scores.append(silhouette_avg)
    
    # Find optimal k based on silhouette score
    optimal_k = range(2, 11)[np.argmax(silhouette_scores)]
    print(f"Optimal number of clusters (silhouette): {optimal_k}")
    print(f"Best silhouette score: {max(silhouette_scores):.3f}")
    
    # Perform clustering with optimal k
    kmeans = KMeans(n_clusters=optimal_k, random_state=ANALYSIS_CONFIG['random_state'], n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    
    # Add cluster labels to dataframe
    df_cluster = df.copy()
    df_cluster['Cluster'] = cluster_labels
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Elbow method plot
    axes[0, 0].plot(k_range, inertias, 'bo-')
    axes[0, 0].set_xlabel('Number of Clusters (k)')
    axes[0, 0].set_ylabel('Inertia')
    axes[0, 0].set_title('Elbow Method for Optimal k')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Silhouette score plot
    axes[0, 1].plot(range(2, 11), silhouette_scores, 'ro-')
    axes[0, 1].set_xlabel('Number of Clusters (k)')
    axes[0, 1].set_ylabel('Silhouette Score')
    axes[0, 1].set_title('Silhouette Analysis')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Clusters in PCA space
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(X_scaled)
    scatter = axes[1, 0].scatter(pca_result[:, 0], pca_result[:, 1], c=cluster_labels, cmap='viridis', alpha=0.6)
    axes[1, 0].set_xlabel('First Principal Component')
    axes[1, 0].set_ylabel('Second Principal Component')
    axes[1, 0].set_title('Clusters in PCA Space')
    plt.colorbar(scatter, ax=axes[1, 0])
    
    # Clusters vs true labels
    cluster_crosstab = pd.crosstab(df_cluster['Cluster'], df_cluster[target_col])
    sns.heatmap(cluster_crosstab, annot=True, fmt='d', cmap='Blues', ax=axes[1, 1])
    axes[1, 1].set_title('Clusters vs True Labels')
    axes[1, 1].set_xlabel('True Labels')
    axes[1, 1].set_ylabel('Clusters')
    
    plt.tight_layout()
    plt.savefig(f"{plots_folder}/clustering_analysis_{target_col}.png", bbox_inches="tight")
    plt.show()
    
    print("\nCluster vs True Labels Cross-tabulation:")
    print(cluster_crosstab)


def statistical_tests(df, target_col='class'):
    """
    Perform statistical tests to identify significant differences between groups
    """
    print("\n" + "="*50)
    print("9. STATISTICAL TESTS")
    print("="*50)
    
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    groups = df.groupby(target_col)
    group_names = list(groups.groups.keys())
    stats_results = []
    
    for col in numerical_cols:
        print(f"\nFeature: {col}")
        print("-" * 20)
        feat_group_data = [df[df[target_col] == name][col].dropna() for name in group_names]
        
        # Test for normality
        normality = []
        for i, data in enumerate(feat_group_data):
            if len(data) > 3:
                _, pval = stats.shapiro(data)
                normality.append(pval > 0.05)
                print(f"  {group_names[i]}: Shapiro p={pval:.4f} ({'Normal' if pval>0.05 else 'Non-normal'})")
            else:
                normality.append(False)
                print(f"  {group_names[i]}: Not enough data for normality test")
        
        # Test for equal variances
        if all([len(x)>3 for x in feat_group_data]):
            _, p_levene = levene(*feat_group_data)
            print(f"  Levene's p={p_levene:.4f} ({'Equal variances' if p_levene>0.05 else 'Unequal variances'})")
        else:
            p_levene = np.nan
        
        # Choose appropriate test
        if all(normality) and (p_levene > 0.05):
            # Use ANOVA if data is normal and variances are equal
            f_stat, p_anova = stats.f_oneway(*feat_group_data)
            print(f"  ANOVA F={f_stat:.3f}, p={p_anova:.4e}")
            test_type = "ANOVA"
            test_stat = f_stat
            p_val = p_anova
        else:
            # Use Kruskal-Wallis if assumptions are violated
            h_stat, p_kruskal = stats.kruskal(*feat_group_data)
            print(f"  Kruskal-Wallis H={h_stat:.3f}, p={p_kruskal:.4e}")
            test_type = "Kruskal-Wallis"
            test_stat = h_stat
            p_val = p_kruskal
        
        stats_results.append({
            'Feature': col,
            'Test': test_type,
            'Test_Statistic': test_stat,
            'P_Value': p_val,
            'Significant': p_val < ANALYSIS_CONFIG['significance_level']
        })
    
    # Create summary table
    results_df = pd.DataFrame(stats_results)
    print("\nSummary Table:")
    print(results_df.round(5))
    
    # Show significant features
    significant = results_df[results_df['Significant']]
    if not significant.empty:
        print(f"\nFeatures with significant class differences (p < {ANALYSIS_CONFIG['significance_level']}):")
        print(significant[['Feature', 'Test', 'P_Value']])
        
        # Create boxplots for significant features
        print("\nBoxplots for features with significant class differences:")
        for col in significant['Feature']:
            plt.figure(figsize=(8, 6))
            sns.boxplot(data=df, x=target_col, y=col)
            plt.title(f'{col} by {target_col} (p={significant[significant["Feature"]==col]["P_Value"].iloc[0]:.4f})')
            plt.tight_layout()
            plt.savefig(f"{plots_folder}/statistical_test_{col}_boxplot_{target_col}.png", bbox_inches="tight")
            plt.show()
    else:
        print(f"\nNo statistically significant feature differences found at p < {ANALYSIS_CONFIG['significance_level']}.")

# Execute immediately
if df is not None:
    clustering_analysis(df, target_col='class')
    statistical_tests(df, target_col='class')
else:
    print("❌ No data available for analysis")

## 📈 Key Findings & Conclusions

### 🔍 **Dataset Quality**
- Clean dataset with no missing values
- 310 patient records with 6 biomechanical features
- Class imbalance present (especially in 3-class problem)

### 📊 **Feature Insights**
- Most features show non-normal distributions
- Outliers present but not excessive
- Strong correlations between some biomechanical measurements

### 🎯 **Classification Insights**
- Binary classification shows better class balance
- Significant differences between groups for most features
- Clear patterns visible in dimensionality reduction plots

### 🔬 **Statistical Findings**
- Multiple features show significant group differences (p < 0.05)
- ANOVA/Kruskal-Wallis tests reveal discriminative power
- Clustering analysis reveals natural groupings

### 💡 **Recommendations for Modeling**
1. **Preprocessing**: Consider standardization due to different scales
2. **Feature Selection**: High VIF features may need attention
3. **Class Imbalance**: SMOTE or other techniques may be beneficial
4. **Model Choice**: Non-linear models may perform better given feature distributions

### 📂 **Generated Outputs**
- All visualizations saved in `plots/` folder
- Detailed analysis report saved as `reports.txt`
- Ready for machine learning pipeline implementation

## 🚀 Running the Complete Analysis

Now we'll execute all analysis functions systematically for both target variables (3-class and binary classification).

In [ ]:
# Optional: Run complete analysis for different target variables
def run_complete_analysis(df, target_variables=['class']):
    """
    Run the complete EDA pipeline for specified target variables
    """
    print("🚀 Running complete EDA analysis...")
    print("="*60)
    
    if df is None:
        print("❌ No data available for analysis")
        return
    
    for target_col in target_variables:
        if target_col not in df.columns:
            print(f"⚠️  Target column '{target_col}' not found in dataset")
            continue
            
        print(f"\n🎯 ANALYZING TARGET: {target_col}")
        print("="*50)
        
        try:
            # Run all analyses for this target
            dataset_overview(df)
            target_analysis(df, target_col=target_col)
            numerical_features_analysis(df, target_col=target_col)
            outlier_analysis(df, target_col=target_col)
            correlation_analysis(df, target_col=target_col)
            feature_relationships(df, target_col=target_col)
            dimensionality_analysis(df, target_col=target_col)
            clustering_analysis(df, target_col=target_col)
            statistical_tests(df, target_col=target_col)
            
            print(f"\n✅ Analysis complete for {target_col}")
            
        except Exception as e:
            print(f"❌ Error analyzing {target_col}: {e}")
    
    print("\n🎉 COMPLETE EDA ANALYSIS FINISHED!")
    print("="*60)
    print(f"📁 All plots saved to: {plots_folder}")
    print("📊 Plots displayed inline in notebook cells")
    print("🔍 Review the plots folder for saved visualizations.")

# Uncomment the line below to run analysis for multiple targets
# run_complete_analysis(df, target_variables=['class', 'binary_class'])

print("\n🔧 Optional batch execution function defined!")
print("📝 Individual cells above run immediately when executed.")
print("⚡ Use run_complete_analysis() for batch processing multiple targets.")